# How to Audit and Mitigate Bias in Machine Learning Models to Improve Fairness

**Problem Statement**:  We are tasked with building a model that can predict whether or not an individual will default on their loan, based on a loan application they have submitted that contains the features listed above.

In practice, the firm has enough resources to review and approve 75% of the applications that are submitted, therefore, they would like to identify the 25% highest risk applications so that they may be either automatically rejected, or perhaps only reviewed if time allows.

The dataset for this notebook comes from the Kaggle dataset "Loan Default Model Trap" (https://www.kaggle.com/jannesklaas/model-trap/version/5).

**Features**
- Minority - 1/0 (1 == Black, 0 == White)
- Sex - 1/0 (1 == Male, 0 == Female)
- ZIP code - categorical
- Rent - 1/0
- Age - continuous
- Education - continuous
- Income - continuous
- Loan_size - continuous
- Payment timing - continuous
- year - discrete, ordinal
- Job stability - continuous
- Occupation - categorical

**Target**
- Default on loan - True / False


## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import make_scorer,confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV
import seaborn as sns
pd.set_option('display.max_columns', None)

In [ ]:
train_raw = pd.read_csv("data/train.csv")
test_raw = pd.read_csv("data/test.csv")

In [ ]:
print(train_raw.shape)
train_raw.head(5)

In [ ]:
print(test_raw.shape)
test_raw.head(5)

## Transform the dataset

In [ ]:
def transformer(data):
    # The variables minority, sex, rent, education, age,i ncome, loan_size, payment_timing, job_stability, year
    #   are continuous
    X = data.filter(items=['minority','sex','rent','education','age','income','loan_size','payment_timing','job_stability','year'])
    # ZIP and occupation must be one-hot encoded
    zip_one_hot = pd.get_dummies(data['ZIP'],drop_first=True)
    X = pd.merge(X,zip_one_hot.add_suffix('_zip'), how='left',left_index=True, right_index=True)
    occupation_one_hot = pd.get_dummies(data['occupation'],drop_first=True)
    X = pd.merge(X,occupation_one_hot.add_suffix('_occupation'), how='left',left_index=True, right_index=True)
    
    # Target will be recode as 1 for True, 0 for False
    y = data['default'].apply(lambda x: 1 if x else 0)
    return X, y

In [ ]:
X_train, y_train = transformer(train_raw)
X_test, y_test = transformer(test_raw)

## Model training and evaluation

We are going to train a logistic regresion model and use a simple grid search (parameter grid can be seen in the code box below on line 7).

For evaluation, it's important that we choose the right metric.  Consider the possible outcomes of our model, given that the intervention automatically denies anyone who is in the top 25% highest predicted scores to default:

- **True positive:**  the model said they would default, and they did
- **False positive:**  the model said they would default, and they didn't
- **True negative:**  the model says they would not default, and they wouldn't
- **False negative:**  the model says they would not default, and they would

Clearly, we want to maximize **True Positives**.  When we consider our two errors, which is more costly?  Arugably, it is the **False Negative**.

Therefore, it makes sense to use a performance metric, that when the model has many **True Positives** will be high, but that will be penalized for haing too many **False Negatives**.  Reference for performance metrics:  https://en.wikipedia.org/wiki/Precision_and_recall.

As a result, we can choose to look to the performance metric recall (also called sensitivity or hit rate), for which the formula is
- **TP / (TP + FN)**


In [ ]:
# Set up
print("Penalty      C         Recall Score")
print("-----------------------------------")
best_recall = 0

# Create the parameter grid and iterate through
grid = [[x,y] for x in ['l1','l2'] for y in [1.00,0.10,0.01]]
for Penalty,c in grid:    
    
    print(f"{Penalty}           {c:.2f}      Training...", end="\r")
    
    # Train a model with the current grid parameters and evaluate the recall score
    temp_model = LogisticRegression(penalty=Penalty,C=c,solver='liblinear',random_state=42).fit(X_train,y_train)
    temp_y_pred_proba = temp_model.predict_proba(X_test)[:,1]
    temp_threshold = np.percentile(temp_y_pred_proba,75)
    temp_y_pred = np.where(temp_y_pred_proba > temp_threshold, 1, 0)
    temp_recall = recall_score(y_test,temp_y_pred)
    
    # Print the results
    print(f"{Penalty}           {c:.2f}      {temp_recall}")
    
    # If this is the best performing model, save the results
    if temp_recall > best_recall:
        best_recall = temp_recall
        model = temp_model
        y_pred = temp_y_pred

# Scikit-learn GridSearchCV implementaiton
# -------------------------------------------
# recall_scorer = make_scorer(recall_score)
# grid = GridSearchCV(LogisticRegression(solver='liblinear'), param_grid={'C': [1, 0.1, 0.01],'penalty':['l1','l2']},scoring=recall_scorer)
# results = grid.fit(X_train,y_train)
# model = grid.estimator

In [ ]:
print(model)

### Inspect the model confusion matrix

In [ ]:
# https://github.com/DTrimarchi10/confusion_matrix/blob/master/cf_matrix.py

cf_matrix = confusion_matrix(y_test, y_pred)
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
categories = ['Comply','Default']
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues',xticklabels=categories,yticklabels=categories)

## Inspect the feature importance

As a naïve approach to evaluating bias, we will apply methods for explainability and interpretability.  For a logistic regression, this means examining the feature importance, or the coefficeint weight of each feature.


In [ ]:
plt.barh(X_train.columns, model.coef_[0], color='b')
plt.tight_layout(-10)
plt.show()

## Bias Audit

Now we will conduct a robust bias audit using the tooklit Aequitas.

This code is a shortened version of the highly recommended Aequitas COMPAS Analysis demo (https://dssg.github.io/aequitas/examples/compas_demo.html).

### Choosing a metric

Consider the context of the problem, and how we may want to think about fairness (consider the "Fairness Tree" from this reference: http://www.datasciencepublicpolicy.org/projects/aequitas/).

In this case, because our intervention is punitive, and we are intervening on a large portion of the population, the Fairness Tree guides us towards choosing the **False Positive Rate** as our fairness metric.

What is the False Positive Rate in context? 

The False Positive group represents those individuals that should have gotten a loan, but were automatically denied.  Through a different lens, one could say these are people who deserved a loan but did not recieve it -- therefore, if our group had bias along the False Positive Rate dimension, that would mean one group was not recieving loans they deserved because of being in that group.

For example, if the False Positive Rate was lower for Males than Females, that would mean the model is more likely predict an individual as defaulting because they are a woman.

In [ ]:
# Run this cell if you need to install aequitas
!pip install aequitas

In [ ]:
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Create a dataframe that is properly formatted for Aequitas
aequitas = X_test.filter(items=["sex","minority"])
aequitas["Sex"] = aequitas["sex"].apply(lambda x: "M" if x == 1 else "F")
aequitas["Race"] = aequitas["minority"].apply(lambda x: "Black" if x == 1 else "White")
aequitas["label_value"] = y_test
aequitas["score"] = y_pred
df = aequitas.drop(columns=['sex','minority'])
df.head()

In [ ]:
df['score'].value_counts()

In [ ]:
protected_attributes = ['Sex','Race']
protected_groups = {'Sex':'F', 'Race':'Black'}

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)

In [ ]:
by_sex = sns.countplot(x="Sex", hue="score", data=df[df.Sex.isin(['M', 'F'])], palette=aq_palette)

In [ ]:
by_race = sns.countplot(x="Race", hue="score", data=df[df.Race.isin(['Black', 'White'])], palette=aq_palette)

In [ ]:
g = Group()
xtab, _ = g.get_crosstabs(df)
absolute_metrics = g.list_absolute_metrics(xtab)
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

In [ ]:
aqp = Plot()
fpr = aqp.plot_group_metric(xtab, 'fpr')

In [ ]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict=protected_groups, alpha=0.05, mask_significance=True)
calculated_disparities = b.list_disparities(bdf)
disparity_significance = b.list_significance(bdf)
bdf[['attribute_name', 'attribute_value'] +  calculated_disparities + disparity_significance]

In [ ]:
hbdf = b.get_disparity_predefined_groups(xtab, original_df=df,
                                         ref_groups_dict=protected_groups,
                                         alpha=0.05,
                                         mask_significance=False)

# View disparity metrics added to dataframe
majority_bdf = b.get_disparity_major_group(xtab, original_df=df, mask_significance=True)
majority_bdf[['attribute_name', 'attribute_value'] +  calculated_disparities + disparity_significance]

In [ ]:
min_metric_bdf = b.get_disparity_min_metric(df=xtab, original_df=df)
# View disparity metrics added to dataframe
min_metric_bdf[['attribute_name', 'attribute_value'] +  calculated_disparities + disparity_significance]

In [ ]:
aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='Sex', significance_alpha=0.05)

In [ ]:
aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='Race', significance_alpha=0.05)

## Bias Mitigation

### A naive approach

In this approach, we will drop the variables Race and Sex and see how it affects bias.

In [ ]:
X_train.columns

In [ ]:
# Attempt 1:  Drop sex and race from the model.
X_train_new = X_train.drop(columns=['minority','sex'])
X_test_new = X_test.drop(columns=['minority','sex'])

In [ ]:
# Set up
print("Penalty      C         Recall Score")
print("-----------------------------------")
best_recall = 0

# Create the parameter grid and iterate through
grid = [[x,y] for x in ['l1','l2'] for y in [1.00,0.10,0.01]]
for Penalty,c in grid:    
    
    print(f"{Penalty}           {c:.2f}      Training...", end="\r")
    
    # Train a model with the current grid parameters and evaluate the recall score
    temp_model = LogisticRegression(penalty=Penalty,C=c,solver='liblinear',random_state=142).fit(X_train_new,y_train)
    temp_y_pred_proba = temp_model.predict_proba(X_test_new)[:,1]
    temp_threshold = np.percentile(temp_y_pred_proba,75)
    temp_y_pred = np.where(temp_y_pred_proba > temp_threshold, 1, 0)
    temp_recall = recall_score(y_test,temp_y_pred)
    
    # Print the results
    print(f"{Penalty}           {c:.2f}      {temp_recall}")
    
    # If this is the best performing model, save the results
    if temp_recall > best_recall:
        best_recall = temp_recall
        model_new = temp_model
        y_pred_new = temp_y_pred

In [ ]:
# Test the performance
# y_pred_proba_new = model_new.predict_proba(X_test_new)[:,1]
# threshold = np.percentile(y_pred_proba_new,75)
# y_pred_new = np.where(y_pred_proba > threshold, 1, 0)

In [ ]:
# Inspect feature importance
plt.barh(X_train_new.columns, model_new.coef_[0], color='b')
plt.tight_layout(-10)
plt.show()

In [ ]:
# Rerurn Aequitas
aequitas = X_test.filter(items=["sex","minority"])
aequitas["Sex"] = aequitas["sex"].apply(lambda x: "M" if x == 1 else "F")
aequitas["Race"] = aequitas["minority"].apply(lambda x: "Black" if x == 1 else "White")
aequitas["label_value"] = y_test
aequitas["score"] = y_pred_new
df = aequitas.drop(columns=['sex','minority'])
df.head()

In [ ]:
by_sex = sns.countplot(x="Sex", hue="score", data=df[df.Sex.isin(['M', 'F'])], palette=aq_palette)

In [ ]:
by_race = sns.countplot(x="Race", hue="score", data=df[df.Race.isin(['Black', 'White'])], palette=aq_palette)

In [ ]:
g = Group()
xtab, _ = g.get_crosstabs(df)
absolute_metrics = g.list_absolute_metrics(xtab)
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
aqp = Plot()
fpr = aqp.plot_group_metric(xtab, 'fpr')

In [ ]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict=protected_groups, alpha=0.05, mask_significance=True)
#calculated_disparities = b.list_disparities(bdf)
#disparity_significance = b.list_significance(bdf)
hbdf = b.get_disparity_predefined_groups(xtab, original_df=df,
                                         ref_groups_dict=protected_groups,
                                         alpha=0.05,
                                         mask_significance=False)

In [ ]:
aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='Sex', significance_alpha=0.05)

In [ ]:
aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='Race', significance_alpha=0.05)

Why did dropping sex and minority have little, if any, affect on the bias in our model?

In [ ]:
X_train['MZ11CD_occupation'].corr(X_train['sex'])

In [ ]:
X_train['MZ11CD_occupation'].corr(X_train['minority'])

**Exercise:**  Run the model again, but without occupation and/or zip code

### Implementation of a robust bias mitigation algorithm

The methods of this analysis are based on the following paper: Rodolfa, K. T., Salomon, E., Haynes, L., Mendieta, I. H., Larson, J., & Ghani, R. (2020, January). Case study: predictive fairness to reduce misdemeanor recidivism through social service interventions. In Proceedings of the 2020 Conference on Fairness, Accountability, and Transparency (pp. 142-153). (https://arxiv.org/abs/2001.09233)

A video summary of the paper can be found here: https://www.youtube.com/watch?v=K55bnPsvFOs

An **in-depth tutorial** on bias mitigation can be found here:  https://www.youtube.com/watch?v=N67pE1AF5cM&list=PLUsfTziJs0NXL0KGEvAf8YU158yrG0JMr&index=2

#### Scenario 1:  The model "as it is"

In [ ]:
y_pred_proba = model.predict_proba(X_test)[:,1]
threshold = np.percentile(y_pred_proba,75)
y_pred = np.where(y_pred_proba > threshold, 1, 0)

In [ ]:
# The following functions are used to evaluate whether or not an observation is
#   a false positive, or true negative
def label_fp(row,pred):
    if (row[pred] == 1 and row['label'] == 0):
        return 1
    else:
        return 0
    
def label_tn(row,pred):
    if (row[pred] == 0 and row['label'] == 0):
        return 1
    else:
        return 0

    
# The following function wil calculate the false negative rate (FNR) of an observation
def calculate_fpr(row):
    return (row['fp']['sum'] / (row['fp']['sum'] + row['tn']['sum']))

In [ ]:
# Variable dictionary
# =====================
#label = True value
#score = Score [0,1] continuous
#pred_k = Assigned risk group based on bias mitigation methods

In [ ]:
#df = X_test.filter(items=["sex","minority"])
df = X_test.copy()
df["Group_x"] = df["minority"].apply(lambda x: "Black" if x == 1 else "White")
df["label"] = y_test
df["score"] = model.predict_proba(X_test)[:,1]
df.head()

In [ ]:
temp = df.copy()
# Calculate the risk outcome based on the score and the IEFP threshold
temp['pred_k'] = temp.apply(lambda row: 1 if row['score'] >= threshold else 0, axis=1)

In [ ]:
# Calculate the fp and tn for all rows
temp['fp'] = temp.apply(lambda row: label_fp(row,'pred_k'), axis=1)
temp['tn'] = temp.apply(lambda row: label_tn(row,'pred_k'), axis=1)
# Count the number of predicted positives, fn, tp, and tn for each group
temp_agg = temp.groupby('Group_x').agg({'Group_x': ['count'],'pred_k': ['sum'],'fp': ['sum'], 'tn': ['sum']})
# Calculate the FPR for each group
temp_agg['fpr'] = temp_agg.apply(calculate_fpr, axis=1)
# Assign the FPR of the Black group as the reference
ref_fpr = temp_agg.loc[temp_agg.index == 'Black']['fpr'][0]
# Calculate the FPR parities of each group
temp_agg['fpr_parity'] = temp_agg.apply(lambda row: row['fpr'] / ref_fpr, axis = 1 )
temp_agg.head()

In [ ]:
# Calculate the model recall score
print(recall_score(temp['label'], temp['pred_k']))

In [ ]:
plt.bar(list(temp_agg.index), temp_agg['fpr'], width=0.4)
plt.ylim(0,1)
plt.ylabel("FPR")

#### Scenario 2:  Equalizing the Bias Metric for all Groups

In this method, we attempt to equalize the FPRs of all grouprs.  This is done by shifting the order in which we select individuals.

In [ ]:
# Create an empty dictionary which will store the assigned group sizes
df_group_x = {"Black": None,
             "White": None}

In [ ]:
# Create a function to calculate the "rolling fpr"
# The rolling fpr is the group FPR as the decision threshold is moved to include
#   additional individuals
def calc_rolling_fpr(row):
    # Call on the global FP and TN variables
    global fp
    global tn
    # If the observation is labeled 1, then we add 1 to the true positives
    #   and we reduce the number of false negatives
    if row['label'] == 0:
        fp+=1
        tn-=1
    # If the label is not 1, then the FNR will not be affected
    # Calculate the rolling FNR
    fpr = (fp / (fp + tn))
    return fpr

# For each group, run the following algorithm
for group in df_group_x:
    # Create a temporary dataframe for the group
    temp_df = df.loc[df['Group_x'] == group]
    
    # Debug: print the group and number of indivudals
    print(f"Calculating rolling FPR for group {group} containing {temp_df.shape[0]} individuals")
    
    # Randomize the order of the dataframe so ties will be broken randomly
    temp_df = temp_df.sample(frac=1).reset_index(drop=True)
    # Sort the values by score in ascending order and reset the indext
    temp_df = temp_df.sort_values(by=['score'], ascending=False)
    temp_df = temp_df.reset_index(drop=True)

    # Initialize the number of true negatives as the number of labeled negatives in the group
    tn = temp_df['label'].value_counts()[0]
    # Initiatlize the number of true positives as 0
    fp = 0
    # Calculate the rolling FNR for each group
    temp_df['FPR_g_i'] = temp_df.apply(lambda row: calc_rolling_fpr(row), axis=1)
    # Create a rolling count of each individual
    temp_df['n_g_i'] = temp_df.index
    # Store the group dataframe
    df_group_x[group] = temp_df

In [ ]:
# Combine all the group dataframes
df_all_groups = pd.concat(list(df_group_x.values()))
print(df_all_groups.shape)

In [ ]:
# Sort the dataframe by FPR ascending and the rolling count ascending
df_all_groups = df_all_groups.sort_values(by=['FPR_g_i','n_g_i'],ascending=[True,True]).reset_index(drop=True)
df_all_groups.head(10)

In [ ]:
# The following code contains the algorithm for equalizing the FPRs

# x is the initial FNR threshold
x = 0
step_size = 0.13
# k stores the final list sizes for each group
k = {'Black': 0, 'White': 0}
# K stores the final list size for all groups (it is the sum over k)
K = 0
# N is the final list size
N = 39000

# If desired, assign a maximum list size for each group
max_list_size = {'White': None, 'Black': None}

# Iteratre while the total list size is less than the desired list size
while K < N:
    # Begin by lowering the FNR threshold by a step of 0.01
    x = x + step_size
    
    # For each group...
    for i in df_all_groups.Group_x.unique():
        # Check that the maximum list size for the group size has not been overstepped
        if (max_list_size[i] == None) or (k[i] < max_list_size[i]):
            # Let k be equal to the maximum individual count above the FNR threshold
            k[i] = df_all_groups.loc[(df_all_groups['Group_x'] == i) & (df_all_groups['FPR_g_i'] < x)]['n_g_i'].max()
    # Calculate k
    K = sum(k.values())

print("The final group sizes are:")
print(k)

In [ ]:
# Select the top k individuals from each group as high risk
df_equal_fpr = df_all_groups
df_equal_fpr['pred_k'] = df_all_groups.apply(lambda row: 1 if row['n_g_i'] < k[row['Group_x']] else 0, axis = 1)

In [ ]:
df_equal_fpr['pred_k'].value_counts()

In [ ]:
temp = df_equal_fpr
# Calculate the fp and tn for all rows
temp['fp'] = temp.apply(lambda row: label_fp(row,'pred_k'), axis=1)
temp['tn'] = temp.apply(lambda row: label_tn(row,'pred_k'), axis=1)
# Count the number of predicted positives, fn, tp, and tn for each group
temp_agg = temp.groupby('Group_x').agg({'Group_x': ['count'],'pred_k': ['sum'],'fp': ['sum'], 'tn': ['sum']})
# Calculate the FPR for each group
temp_agg['fpr'] = temp_agg.apply(calculate_fpr, axis=1)
# Assign the FPR of the Black group as the reference
ref_fpr = temp_agg.loc[temp_agg.index == 'Black']['fpr'][0]
# Calculate the FPR parities of each group
temp_agg['fpr_parity'] = temp_agg.apply(lambda row: row['fpr'] / ref_fpr, axis = 1 )
temp_agg.head()

In [ ]:
print(recall_score(temp['label'], temp['pred_k']))

In [ ]:
# Plot the FNR for each group
plt.bar(list(temp_agg.index), temp_agg['fpr'], width=0.4)
plt.ylim(0,1)
plt.ylabel("FPR")

**Exercise:** see how this mitigation affected gender